In [1]:
# Web_scraping.py

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd


In [2]:
def get_data(link):
  get_url = link
  content = urlopen(get_url)
  page_soup = soup(content.read(),"html.parser")
  content.close()

  name = page_soup.find('h1',{"class":"_3aqeL"})
  location = page_soup.find('div',{"class":"Gf2NS _2Y6HW"})
  rating = page_soup.find('div',{"class":"_2l3H5"})
  number_of_rating = page_soup.find('span',{"class":"_1iYuU"})
  
  Food_items=[]  #List to store name of the food items
  prices=[]      #List to store prices of the food itmes
  categories = []
  descriptions = []
  restaurent_name = []
  locations = []
  ratings = []
  num_ratings = []
  containers = page_soup.findAll("div", attrs={"class":"_2dS-v"})
  for x in containers :
    category = x.find('h2',{"class":"M_o7R _27PKo"}) # getting food categories
    mini_containers = x.findAll("div",{"class":"_2wg_t"})
    for j in mini_containers :
      item = j.find("div",{"class":"styles_itemName__hLfgz"}) # getting here food item names
      price = j.find("span",{"class":"rupee"}) # getting here food prices
      item_des = j.find("div",{"class":"styles_itemDesc__3vhM0"}) # getting description
      
      restaurent_name.append(name.text)
      locations.append(location.text)
      ratings.append(rating.text)
      num_ratings.append(number_of_rating.text)
      try :
        categories.append(category.text)
      except AttributeError :
        categories.append(None)

      try :
        prices.append(price.text)
      except AttributeError :
        pass

      try :
        Food_items.append(item.text) 
      except AttributeError :
        pass

      try :
        descriptions.append(item_des.text)
      except AttributeError :
        descriptions.append(None)
  df = pd.DataFrame({'restaurent_name':restaurent_name,'location':locations,
                     'ratings':ratings , 'number_of_rating' : num_ratings,
                     'Food Item Name':Food_items,'item_prices':prices ,
                     'item_description':descriptions,'category':categories          
                     }) 
  return df        


Start Scrapping any Top 3 Restaurants from Swiggy Platform. Scrape Following
things - Name, Area, Ratings, No of Ratings, Item Name, Item Category, Item Price,
Item Description, Tags(Bestseller).


In [3]:
links = ["https://www.swiggy.com/restaurants/the-belgian-waffle-co-v-v-puram-shankarapura-bangalore-197238",
         "https://www.swiggy.com/restaurants/burger-king-opposite-to-west-side-shivajinagar-bangalore-5938",
         "https://www.swiggy.com/restaurants/polar-bear-vv-puram-shankarapura-bangalore-5893"]
frames = []         
for link in links :
  data_frame =  get_data(link)
  frames.append(data_frame)  

result = pd.concat(frames)        
result.to_csv('menu.csv', index=False, encoding='utf-8')


Data cleaning 

In [7]:
import pandas as pd
df = result
df = df[pd.notnull(df['item_description'])]
result = df

In [8]:
result.to_csv('menu.csv', index=False, encoding='utf-8')

In [15]:
import re
import string
import nltk

import pandas as pd

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [17]:

nltk.download('punkt')
nltk.download('stopwords')

STOP_WORDS = stopwords.words()

# removing the emojies
# https://www.kaggle.com/alankritamishra/covid-19-tweet-sentiment-analysis#Sentiment-analysis
EMOJI_PATTERN = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)


def cleaning(text):
    """
    Convert to lowercase.
    Rremove URL links, special characters and punctuation.
    Tokenize and remove stop words.
    """
    text = text.lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[’“”…]', '', text)

    text = EMOJI_PATTERN.sub(r'', text)

    # removing the stop-words
    text_tokens = word_tokenize(text)
    tokens_without_sw = [
        word for word in text_tokens if not word in STOP_WORDS]
    filtered_sentence = (" ").join(tokens_without_sw)
    text = filtered_sentence

    return text


if __name__ == "__main__":
    max_rows = 1000  # 'None' to read whole file
    input_file = 'menu.csv'
    df = pd.read_csv(input_file,
                     delimiter = ',',
                     nrows = max_rows,
                     engine = "python")
 
 ###### frequent words in Item Descrition ###########
    dt = df['item_description'].apply(cleaning)
    word_count = Counter(" ".join(dt).split()).most_common(10)
    print("###### frequent words in Item Descrition ###########")
    word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
    print(word_frequency)
     
###### frequent words in Item name ###########

    dt = df['Food Item Name'].apply(cleaning)
    word_count = Counter(" ".join(dt).split()).most_common(10)
    print("###### frequent words in Item Name ###########")
    word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
    print(word_frequency)
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
###### frequent words in Item Descrition ###########
        Word  Frequency
0  chocolate        256
1     waffle        126
2      cream        103
3     crispy         99
4          1         87
5    chicken         87
6      patty         84
7        ice         80
8      fries         73
9    classic         69
###### frequent words in Item Name ###########
        Word  Frequency
0    chicken         77
1  chocolate         73
2     waffle         65
3      combo         55
4          1         48
5        veg         47
6    whopper         46
7      fries         34
8     burger         31
9        pcs         29


Follow this link for visualization
https://datastudio.google.com/reporting/84def3d4-0c8e-4139-adf1-cdabb3050120